# Ocean Genomic Analysis with Myria!

Courtesy of the Chisholm Lab and the Simons Foundation, we have data from ocean expeditions conducted in 2010-2011. In these expeditions, ships stopped at various sites in the Atlantic ocean and gathered the following:

1. (Small) Environmental measurements taken at many depths at each site.
2. (Large) Genomic sequnce data taken at a couple depths at each site.

In this notebook we sketch the following pipeline of analysis to gain insight into the microbial life present in the ocean, as evidenced from the sequenced samples, and how the microbial life relates to the environment:

1. Connect to Myria and ingest the datasets of extracted kmers.
2. Combine samples into a single relation for unified analysis. Normalize.
3. Compute **diversity measures** of each sample.
4. Compute **Bray-Curtis distances** between samples.
5. Pre-process and upload **environmental data**.
6. Link environmental data to the genomic samples via **interpolation**.
7. Exploratory analysis with t-SNE and degree distributions.

The result of our pipeline feeds the visualizations hosted at our istcdemo screen.

# Establish Connection to Myria

In [27]:
from myria import *
import numpy
%load_ext myria

In [50]:
connection = MyriaConnection(rest_url='http://node-109:8753', \
                             execution_url = 'http://node-111:8124')
MyriaRelation.DefaultConnection = connection

## View Relations in Database

In [129]:
datasets = connection.datasets()
print len(datasets)
for d in datasets[0:8]:
    print d['relationKey']['relationName']

426
AboveNotBelow
BC_complete
BC_condensed
BelowNotAbove
EnvironmentalData
GeotracesConversionBystation
Kmer11_gc_content
SampleToEnvironmental


# Insert kmer Datasets

The following directory contains the extracted 11-mers of all overlapped samples.

In [23]:
dir_parsed_11_ol = '/home/gridsan/groups/istcdata/datasets/ocean_metagenome/csv_data/parsed_11_cnt'
from os import listdir
from os.path import isfile, join
files_parsed_11_ol = [f for f in listdir(dir_parsed_11_ol) \
                      if isfile(join(dir_parsed_11_ol, f)) and f[-4:] == '.csv']

In [49]:
print len(files_parsed_11_ol)
#print join(dir_parsed_11_ol, files_parsed_11_ol[0])

396


The following code inserts each sample as a separate relation in the database.

In [51]:
for f in files_parsed_11_ol:
    fpath = 'file://%s' % join(dir_parsed_11_ol, f)
    rname = 'kmercnt_11_forward_%s_ol' % f[:5]
    query = %myrial T = load("@fpath", csv(schema(kmer:string, cnt:int))); store(T, @rname);
    print '%s: %s' % (f, query.status)
    break # Comment this line to insert all datasets, not just the first one.

S0168_11_cnt.csv: SUCCESS


## Union samples into one relation

The following code outlines a simple way of automatically union'ing together all the relations in Myria that match a regular expression. We used this technique alongside batching the union queries.

In [124]:
pattern = re.compile('^kmercnt_11_rc_(S\d\d\d\d)_ol$')
resultRelation = 'kmercnt_11_rc_ol'

counter = 0
s = ''
for d in datasets:
    rn = d['relationKey']['relationName']
    m = pattern.match(rn)
    if m:
        sid = m.group(1)
        if s == '': s = '{1} = scan({0}); R = [from {1} emit "{1}" as sampleid, kmer, cnt];\n'.format(rn, sid)
        else: s += '{1} = scan({0}); R = R + [from {1} emit "{1}" as sampleid, kmer, cnt];\n'.format(rn, sid)
s += 'store(R, %s);' % resultRelation
print s
# Uncomment these lines to execute the query
#query  = %myrial @s
#print 'Query {}: {}'.format(query.query_id, query.status)

S0001 = scan(kmercnt_11_rc_S0001_ol); R = [from S0001 emit "S0001" as sampleid, kmer, cnt];
S0002 = scan(kmercnt_11_rc_S0002_ol); R = R + [from S0002 emit "S0002" as sampleid, kmer, cnt];
S0003 = scan(kmercnt_11_rc_S0003_ol); R = R + [from S0003 emit "S0003" as sampleid, kmer, cnt];
S0004 = scan(kmercnt_11_rc_S0004_ol); R = R + [from S0004 emit "S0004" as sampleid, kmer, cnt];
S0005 = scan(kmercnt_11_rc_S0005_ol); R = R + [from S0005 emit "S0005" as sampleid, kmer, cnt];
S0006 = scan(kmercnt_11_rc_S0006_ol); R = R + [from S0006 emit "S0006" as sampleid, kmer, cnt];
S0008 = scan(kmercnt_11_rc_S0008_ol); R = R + [from S0008 emit "S0008" as sampleid, kmer, cnt];
S0009 = scan(kmercnt_11_rc_S0009_ol); R = R + [from S0009 emit "S0009" as sampleid, kmer, cnt];
S0010 = scan(kmercnt_11_rc_S0010_ol); R = R + [from S0010 emit "S0010" as sampleid, kmer, cnt];
S0011 = scan(kmercnt_11_rc_S0011_ol); R = R + [from S0011 emit "S0011" as sampleid, kmer, cnt];
S0012 = scan(kmercnt_11_rc_S0012_ol); R = R 

## Sum together overlapped and non-overlapped kmers

In [ ]:
%%query
ol = scan(kmercnt_11_forward_ol_Pkmer);
nol = scan(kmercnt_11_forward_nol_Pkmer);

olnol = select sampleid, kmer, o.cnt + o2.cnt as cnt
            from ol o, nol o2
            where o.kmer = o2.kmer
            and o.sampleid = o2.sampleid;

store(olnol, kmercnt_11_forward_Pkmer, [kmer]);

## Compute degree table

In [ ]:
%%query 
T = scan(kmercnt_11_forward_Pkmer);
sum_cnt = select sampleid, sum(cnt) as sum_cnt from T;
store(sum_cnt, kmercnt_11_sum_Psample, [sampleid]);

## Normalize kmer counts

Because some samples have more sequences than others, we normalize each kmer count by the total number of kmers in that sequence. The result is the frequency each kmer appears at.

In [ ]:
%%query
Tcnt = scan(kmercnt_11_forward_Pkmer);
Tsum = scan(kmercnt_11_sum_Psample);
Tnorm = select sampleid, kmer, cnt / sum_cnt as norm_cnt
    from Tcnt c, Tsum s
    where c.sampleid = s.sampleid;
store(Tnorm, kmercntnorm_11_forward_Pkmer, [kmer]);

# Compute diversity measures on kmer frequencies

The [**Simpson diversity**](https://en.wikipedia.org/wiki/Diversity_index#Simpson_index) is the probability that two kmers selected at random from the sample represent are the same.

The **Entropy skew** is how far away the kmer distribution is from the maximum possible entropy, which is 22 bits for 11-mers.

We also compute the mean, count, and standard deviation of the normalized kmer counts for each sample.

In [ ]:
%%query
-- K=11: 4^11 = 4194304
T1 = scan(kmercntnorm_11_forward_Pkmer);
TMean = select sampleid, sum(norm_cnt) / 4194304 as mean, 
		count(*) as nnz, sum(norm_cnt * norm_cnt) as simp,
		22 + sum(norm_cnt*log(norm_cnt)/log(2)) as entropy
	 	from T1;
T3 = select t.sampleid, tm.mean, tm.nnz
           (t.norm_cnt - tm.mean)*(t.norm_cnt - tm.mean) as stdevi,
           tm.simp, tm.entropy
      where t.sampleid = tm.sampleid
     from T1 t, TMean tm;
store(T3, kmercntnorm_11_forward_stats, [sampleid]);

# Compute Bray-Curtis Distances between samples

The [Bray-Curtis dissimilarity](https://en.wikipedia.org/wiki/Bray%E2%80%93Curtis_dissimilarity) is a common metric used in ecology to compare the composition of species at different sites.  

We run Bray-Curtis on the normalized kmer frequencies via the formula `1 - sum(min(a,b))`
where `a` and `b` are the frequencies from two samples for a particular kmer. The sum is over all 4^11 kmers.
We compute this for every pair of samples. In the visualization, we only display the ones we have environmental data for.

In [ ]:
%%query
k = scan(kmercntnorm_11_forward_Pkmer);
kmers = select * from k;

X = select a.sampleid as asample, b.sampleid as bsample,
    case when a.norm_cnt < b.norm_cnt then a.norm_cnt else b.norm_cnt end as minv
    from kmers a, kmers b
    where a.kmer = b.kmer
    and a.sampleid < b.sampleid;

Y = select asample, bsample, 1 - sum(minv) as BCdis
    from X;
store(Y, BC_full);

# Environmental Data

## Pre-processing and ingest

The following file provides the location and pressure value at which each sample was collected.

In [52]:
%%query
C = load("file:///home/gridsan/dhutchison/gits/istc_oceanography/metadata/geotraces_conversion_bystation_v2.tsv",
csv(schema(Library:string, Cruise:string, Station:int, BODC:int, CTDPRS:float, Date:string, Longitude:float, Latitude:float),skip=1,delimiter="\t"));
store(C, GeotracesConversionBystation);

,BODC,CTDPRS,Cruise,Date,Latitude,Library,Longitude,Station
0,1608554,10.0,GA02,2010-05-10T10:40:29.000,49.72200,S0001,317.55310,10
1,1608554,40.0,GA02,2010-05-10T10:40:29.000,49.72200,S0002,317.55310,10
2,1608554,80.0,GA02,2010-05-10T10:40:29.000,49.72200,S0003,317.55310,10
3,1608554,100.0,GA02,2010-05-10T10:40:29.000,49.72200,S0004,317.55310,10
4,1608554,160.0,GA02,2010-05-10T10:40:29.000,49.72200,S0005,317.55310,10
5,1608554,180.0,GA02,2010-05-10T10:40:29.000,49.72200,S0006,317.55310,10
6,1608603,10.0,GA02,2010-05-12T18:57:00.000,46.31190,S0008,320.34161,12
7,1608603,40.0,GA02,2010-05-12T18:57:00.000,46.31190,S0009,320.34161,12
8,1608603,80.0,GA02,2010-05-12T18:57:00.000,46.31190,S0010,320.34161,12
9,1608603,100.0,GA02,2010-05-12T18:57:00.000,46.31190,S0011,320.34161,12


The environmental data is quite small, compared to the genomic data. Many of the columns are sparse measurements; not every variable was measured at every depth of every site.

The following code replaces missing values with 0s. It treats actual 0s as small values 0.000001s.
Our choice of 0 facilitates the interpolation we run below when we link the environmental data to the genomic data, which was taken at a particular depth not necessarily equal to the depth environmental data was recorded.

In [94]:
import pandas as pd
data = pd.read_csv("file:///home/gridsan/dhutchison/gits/istc_oceanography/metadata/GA02_IDP2014_v2_Discrete_Sample_Data_cut.csv")


In [95]:
data = data.replace(0,0.000001) #Replace real 0s with an epsilon
#sum(data2["NITRAT [umol/kg]"] == 0.000001)
data = data.fillna(0) # Replace missing values with 0
data["Bottle Number"] = data["Bottle Number"].astype(int) #ensure all values are ints
zero_eps_file = "GA02_IDP2014_v2_Discrete_Sample_Data_cut_zero_eps.csv"
data.to_csv(zero_eps_file)

In [96]:
#file:///home/gridsan/dhutchison/gits/istc_oceanography/metadata/GA02_IDP2014_v2_Discrete_Sample_Data_cut_zero_eps.csv
import os
filepath = "file://%s" % join(os.getcwd(), zero_eps_file)
query = %myrial E = load("@filepath", \
                         csv(schema(rowid:int,Cruise:string,Station:int,Date:string,Longitude:float,Latitude:float,BotDepth:int,OperatorCruiseName:string, CTDPRS:float,CastId:string,SamplingDevice:string,BottleNumber:int,BODCBottleNumber:int,Depth:int,CTDTMP:float,CTDSAL:float, \
                                    OXYGEN:float,CTDOXY:float,PHSPHT:float,SILCAT:float,NITRAT:float,NITRIT:float,TALK:float,DIC:float,H2O_2_D_DELTA_BOTTLE:float,H2O_18_D_DELTA_BOTTLE:float,Al_D_CONC_BOTTLE:float,STANDARD_DEV:float,Ba_D_CONC_BOTTLE:float,Cd_D_CONC_BOTTLE:float,Fe_D_CONC_BOTTLE:float,Fe_D_CONC_BOTTLE_FIA:float,Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV:float,Mn_D_CONC_BOTTLE:float,Mn_D_CONC_BOTTLE_STANDARD_DEV:float,Mo_D_CONC_BOTTLE:float,Ni_D_CONC_BOTTLE:float,Pb_D_CONC_BOTTLE:float,U_D_CONC_BOTTLE:float,Zn_D_CONC_BOTTLE:float,Y_D_CONC_BOTTLE:float,La_D_CONC_BOTTLE:float,Pa_231_D_CONC_BOTTLE:float,Pa_231_D_CONC_BOTTLE_STANDARD_DEV:float,Th_230_D_CONC_BOTTLE:float,Th_230_D_CONC_BOTTLE_STANDARD_DEV:float,Th_232_D_CONC_BOTTLE:float,Th_232_D_CONC_BOTTLE_STANDARD_DEV:float,Th_234_D_CONC_BOTTLE:float,Th_234_D_CONC_BOTTLE_STANDARD_DEV:float),skip=1,delimiter=",")); \
store(E, EnvironmentalData, [Station]);
print query.status

SUCCESS
<function submit at 0x32088c0>


## Interpolate environmental data to samples

Interpolation is awkward but doable in sql-like languages. There is a paper under review which proposes a new relational operator---interpolating join---to handle different kinds of interpolation on data of arbitrary dimension.

Here is the logic for each sampleid:

* (Majority case)  If there is an environmental data row for a pressure both above and below the pressure at which the sample was taken, then linearly interpolate between those two surrounding rows.
* (10 samples taken at shallow depths fall into this case)  If there is an environmental data row for a pressure below the pressure at which the sample was taken but no row above, then assume the point above (nearest neighbor strategy).
* No samples fell into the opposite case: having a point above but no point below.
* We have no envionmental data for half the samples.  This is because we are missing conversion data for cruise GA03; i.e. we do not know where and at what depth the samples from cruise GA03 were taken.

Here is the logic for each attribute (OXYGEN, SILCAT, PHSPHT, etc.), assuming there is a row above and below:

* If the closest point above and below both have measurements, then interpolate between them.
* If one of the closest points above or below is missing, then use the value of the one that is not missing.
* Otherwise leave the data point as missing (a zero).

In [97]:
%%query 
-- This query does linear interpolation on pressure (CTDPRS)
-- between the conversion table C and the environmental metadata table E
-- Extrapolation is limited to the nearest point
C = scan(GeotracesConversionBystation);
E = scan(EnvironmentalData);

-- get the greatest pressure of E that is below C
Below1 = select c.CTDPRS as pressure, c.Library, c.Station,
    max(e.CTDPRS) as pressureBelow
from C c, E e
where c.Station = e.Station and e.CTDPRS <= c.CTDPRS
and e.CTDPRS != 0;

-- get the average of metadata from E that is at that pressure
Below2 = select b.pressure, b.Library, b.Station, b.pressureBelow,
    case when sum(case when e.Depth != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Depth) / sum(case when e.Depth != 0 then 1 else 0 end) end as depthBelow, 
 case when sum(case when e.CTDTMP != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.CTDTMP) / sum(case when e.CTDTMP != 0 then 1 else 0 end) end as tempBelow, 
 case when sum(case when e.CTDTMP != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.CTDSAL) / sum(case when e.CTDTMP != 0 then 1 else 0 end) end as salBelow,
case when sum(case when e.OXYGEN != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.OXYGEN) / sum(case when e.OXYGEN != 0 then 1 else 0 end) end as OXYGEN_Below,case when sum(case when e.CTDOXY != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.CTDOXY) / sum(case when e.CTDOXY != 0 then 1 else 0 end) end as CTDOXY_Below,case when sum(case when e.PHSPHT != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.PHSPHT) / sum(case when e.PHSPHT != 0 then 1 else 0 end) end as PHSPHT_Below,case when sum(case when e.SILCAT != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.SILCAT) / sum(case when e.SILCAT != 0 then 1 else 0 end) end as SILCAT_Below,case when sum(case when e.NITRAT != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.NITRAT) / sum(case when e.NITRAT != 0 then 1 else 0 end) end as NITRAT_Below,case when sum(case when e.NITRIT != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.NITRIT) / sum(case when e.NITRIT != 0 then 1 else 0 end) end as NITRIT_Below,case when sum(case when e.TALK != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.TALK) / sum(case when e.TALK != 0 then 1 else 0 end) end as TALK_Below,case when sum(case when e.DIC != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.DIC) / sum(case when e.DIC != 0 then 1 else 0 end) end as DIC_Below,case when sum(case when e.H2O_2_D_DELTA_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.H2O_2_D_DELTA_BOTTLE) / sum(case when e.H2O_2_D_DELTA_BOTTLE != 0 then 1 else 0 end) end as H2O_2_D_DELTA_BOTTLE_Below,case when sum(case when e.H2O_18_D_DELTA_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.H2O_18_D_DELTA_BOTTLE) / sum(case when e.H2O_18_D_DELTA_BOTTLE != 0 then 1 else 0 end) end as H2O_18_D_DELTA_BOTTLE_Below,case when sum(case when e.Al_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Al_D_CONC_BOTTLE) / sum(case when e.Al_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Al_D_CONC_BOTTLE_Below,case when sum(case when e.STANDARD_DEV != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.STANDARD_DEV) / sum(case when e.STANDARD_DEV != 0 then 1 else 0 end) end as STANDARD_DEV_Below,case when sum(case when e.Ba_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Ba_D_CONC_BOTTLE) / sum(case when e.Ba_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Ba_D_CONC_BOTTLE_Below,case when sum(case when e.Cd_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Cd_D_CONC_BOTTLE) / sum(case when e.Cd_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Cd_D_CONC_BOTTLE_Below,case when sum(case when e.Fe_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Fe_D_CONC_BOTTLE) / sum(case when e.Fe_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Fe_D_CONC_BOTTLE_Below,case when sum(case when e.Fe_D_CONC_BOTTLE_FIA != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Fe_D_CONC_BOTTLE_FIA) / sum(case when e.Fe_D_CONC_BOTTLE_FIA != 0 then 1 else 0 end) end as Fe_D_CONC_BOTTLE_FIA_Below,case when sum(case when e.Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV) / sum(case when e.Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV != 0 then 1 else 0 end) end as Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV_Below,case when sum(case when e.Mn_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Mn_D_CONC_BOTTLE) / sum(case when e.Mn_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Mn_D_CONC_BOTTLE_Below,case when sum(case when e.Mn_D_CONC_BOTTLE_STANDARD_DEV != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Mn_D_CONC_BOTTLE_STANDARD_DEV) / sum(case when e.Mn_D_CONC_BOTTLE_STANDARD_DEV != 0 then 1 else 0 end) end as Mn_D_CONC_BOTTLE_STANDARD_DEV_Below,case when sum(case when e.Mo_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Mo_D_CONC_BOTTLE) / sum(case when e.Mo_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Mo_D_CONC_BOTTLE_Below,case when sum(case when e.Ni_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Ni_D_CONC_BOTTLE) / sum(case when e.Ni_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Ni_D_CONC_BOTTLE_Below,case when sum(case when e.Pb_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Pb_D_CONC_BOTTLE) / sum(case when e.Pb_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Pb_D_CONC_BOTTLE_Below,case when sum(case when e.U_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.U_D_CONC_BOTTLE) / sum(case when e.U_D_CONC_BOTTLE != 0 then 1 else 0 end) end as U_D_CONC_BOTTLE_Below,case when sum(case when e.Zn_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Zn_D_CONC_BOTTLE) / sum(case when e.Zn_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Zn_D_CONC_BOTTLE_Below,case when sum(case when e.Y_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Y_D_CONC_BOTTLE) / sum(case when e.Y_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Y_D_CONC_BOTTLE_Below,case when sum(case when e.La_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.La_D_CONC_BOTTLE) / sum(case when e.La_D_CONC_BOTTLE != 0 then 1 else 0 end) end as La_D_CONC_BOTTLE_Below,case when sum(case when e.Pa_231_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Pa_231_D_CONC_BOTTLE) / sum(case when e.Pa_231_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Pa_231_D_CONC_BOTTLE_Below,case when sum(case when e.Pa_231_D_CONC_BOTTLE_STANDARD_DEV != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Pa_231_D_CONC_BOTTLE_STANDARD_DEV) / sum(case when e.Pa_231_D_CONC_BOTTLE_STANDARD_DEV != 0 then 1 else 0 end) end as Pa_231_D_CONC_BOTTLE_STANDARD_DEV_Below,case when sum(case when e.Th_230_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Th_230_D_CONC_BOTTLE) / sum(case when e.Th_230_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Th_230_D_CONC_BOTTLE_Below,case when sum(case when e.Th_230_D_CONC_BOTTLE_STANDARD_DEV != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Th_230_D_CONC_BOTTLE_STANDARD_DEV) / sum(case when e.Th_230_D_CONC_BOTTLE_STANDARD_DEV != 0 then 1 else 0 end) end as Th_230_D_CONC_BOTTLE_STANDARD_DEV_Below,case when sum(case when e.Th_232_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Th_232_D_CONC_BOTTLE) / sum(case when e.Th_232_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Th_232_D_CONC_BOTTLE_Below,case when sum(case when e.Th_232_D_CONC_BOTTLE_STANDARD_DEV != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Th_232_D_CONC_BOTTLE_STANDARD_DEV) / sum(case when e.Th_232_D_CONC_BOTTLE_STANDARD_DEV != 0 then 1 else 0 end) end as Th_232_D_CONC_BOTTLE_STANDARD_DEV_Below,case when sum(case when e.Th_234_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Th_234_D_CONC_BOTTLE) / sum(case when e.Th_234_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Th_234_D_CONC_BOTTLE_Below,case when sum(case when e.Th_234_D_CONC_BOTTLE_STANDARD_DEV != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Th_234_D_CONC_BOTTLE_STANDARD_DEV) / sum(case when e.Th_234_D_CONC_BOTTLE_STANDARD_DEV != 0 then 1 else 0 end) end as Th_234_D_CONC_BOTTLE_STANDARD_DEV_Below
from Below1 b, E e
where b.Station = e.Station and b.pressureBelow = e.CTDPRS;

-- get the least pressure of E that is above C
Above1 = select c.CTDPRS as pressure, c.Library, c.Station,
    min(e.CTDPRS) as pressureAbove
from C c, E e
where c.Station = e.Station and e.CTDPRS >= c.CTDPRS
and c.CTDPRS != -1;

-- get the average of metadata from E that is at that pressure
Above2 = select a.pressure, a.Library, a.Station, a.pressureAbove, 
     case when sum(case when e.Depth != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Depth) / sum(case when e.Depth != 0 then 1 else 0 end) end as depthAbove, 
 case when sum(case when e.CTDTMP != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.CTDTMP) / sum(case when e.CTDTMP != 0 then 1 else 0 end) end as tempAbove, 
 case when sum(case when e.CTDTMP != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.CTDSAL) / sum(case when e.CTDTMP != 0 then 1 else 0 end) end as salAbove,
case when sum(case when e.OXYGEN != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.OXYGEN) / sum(case when e.OXYGEN != 0 then 1 else 0 end) end as OXYGEN_Above,case when sum(case when e.CTDOXY != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.CTDOXY) / sum(case when e.CTDOXY != 0 then 1 else 0 end) end as CTDOXY_Above,case when sum(case when e.PHSPHT != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.PHSPHT) / sum(case when e.PHSPHT != 0 then 1 else 0 end) end as PHSPHT_Above,case when sum(case when e.SILCAT != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.SILCAT) / sum(case when e.SILCAT != 0 then 1 else 0 end) end as SILCAT_Above,case when sum(case when e.NITRAT != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.NITRAT) / sum(case when e.NITRAT != 0 then 1 else 0 end) end as NITRAT_Above,case when sum(case when e.NITRIT != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.NITRIT) / sum(case when e.NITRIT != 0 then 1 else 0 end) end as NITRIT_Above,case when sum(case when e.TALK != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.TALK) / sum(case when e.TALK != 0 then 1 else 0 end) end as TALK_Above,case when sum(case when e.DIC != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.DIC) / sum(case when e.DIC != 0 then 1 else 0 end) end as DIC_Above,case when sum(case when e.H2O_2_D_DELTA_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.H2O_2_D_DELTA_BOTTLE) / sum(case when e.H2O_2_D_DELTA_BOTTLE != 0 then 1 else 0 end) end as H2O_2_D_DELTA_BOTTLE_Above,case when sum(case when e.H2O_18_D_DELTA_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.H2O_18_D_DELTA_BOTTLE) / sum(case when e.H2O_18_D_DELTA_BOTTLE != 0 then 1 else 0 end) end as H2O_18_D_DELTA_BOTTLE_Above,case when sum(case when e.Al_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Al_D_CONC_BOTTLE) / sum(case when e.Al_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Al_D_CONC_BOTTLE_Above,case when sum(case when e.STANDARD_DEV != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.STANDARD_DEV) / sum(case when e.STANDARD_DEV != 0 then 1 else 0 end) end as STANDARD_DEV_Above,case when sum(case when e.Ba_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Ba_D_CONC_BOTTLE) / sum(case when e.Ba_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Ba_D_CONC_BOTTLE_Above,case when sum(case when e.Cd_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Cd_D_CONC_BOTTLE) / sum(case when e.Cd_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Cd_D_CONC_BOTTLE_Above,case when sum(case when e.Fe_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Fe_D_CONC_BOTTLE) / sum(case when e.Fe_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Fe_D_CONC_BOTTLE_Above,case when sum(case when e.Fe_D_CONC_BOTTLE_FIA != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Fe_D_CONC_BOTTLE_FIA) / sum(case when e.Fe_D_CONC_BOTTLE_FIA != 0 then 1 else 0 end) end as Fe_D_CONC_BOTTLE_FIA_Above,case when sum(case when e.Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV) / sum(case when e.Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV != 0 then 1 else 0 end) end as Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV_Above,case when sum(case when e.Mn_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Mn_D_CONC_BOTTLE) / sum(case when e.Mn_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Mn_D_CONC_BOTTLE_Above,case when sum(case when e.Mn_D_CONC_BOTTLE_STANDARD_DEV != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Mn_D_CONC_BOTTLE_STANDARD_DEV) / sum(case when e.Mn_D_CONC_BOTTLE_STANDARD_DEV != 0 then 1 else 0 end) end as Mn_D_CONC_BOTTLE_STANDARD_DEV_Above,case when sum(case when e.Mo_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Mo_D_CONC_BOTTLE) / sum(case when e.Mo_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Mo_D_CONC_BOTTLE_Above,case when sum(case when e.Ni_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Ni_D_CONC_BOTTLE) / sum(case when e.Ni_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Ni_D_CONC_BOTTLE_Above,case when sum(case when e.Pb_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Pb_D_CONC_BOTTLE) / sum(case when e.Pb_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Pb_D_CONC_BOTTLE_Above,case when sum(case when e.U_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.U_D_CONC_BOTTLE) / sum(case when e.U_D_CONC_BOTTLE != 0 then 1 else 0 end) end as U_D_CONC_BOTTLE_Above,case when sum(case when e.Zn_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Zn_D_CONC_BOTTLE) / sum(case when e.Zn_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Zn_D_CONC_BOTTLE_Above,case when sum(case when e.Y_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Y_D_CONC_BOTTLE) / sum(case when e.Y_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Y_D_CONC_BOTTLE_Above,case when sum(case when e.La_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.La_D_CONC_BOTTLE) / sum(case when e.La_D_CONC_BOTTLE != 0 then 1 else 0 end) end as La_D_CONC_BOTTLE_Above,case when sum(case when e.Pa_231_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Pa_231_D_CONC_BOTTLE) / sum(case when e.Pa_231_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Pa_231_D_CONC_BOTTLE_Above,case when sum(case when e.Pa_231_D_CONC_BOTTLE_STANDARD_DEV != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Pa_231_D_CONC_BOTTLE_STANDARD_DEV) / sum(case when e.Pa_231_D_CONC_BOTTLE_STANDARD_DEV != 0 then 1 else 0 end) end as Pa_231_D_CONC_BOTTLE_STANDARD_DEV_Above,case when sum(case when e.Th_230_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Th_230_D_CONC_BOTTLE) / sum(case when e.Th_230_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Th_230_D_CONC_BOTTLE_Above,case when sum(case when e.Th_230_D_CONC_BOTTLE_STANDARD_DEV != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Th_230_D_CONC_BOTTLE_STANDARD_DEV) / sum(case when e.Th_230_D_CONC_BOTTLE_STANDARD_DEV != 0 then 1 else 0 end) end as Th_230_D_CONC_BOTTLE_STANDARD_DEV_Above,case when sum(case when e.Th_232_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Th_232_D_CONC_BOTTLE) / sum(case when e.Th_232_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Th_232_D_CONC_BOTTLE_Above,case when sum(case when e.Th_232_D_CONC_BOTTLE_STANDARD_DEV != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Th_232_D_CONC_BOTTLE_STANDARD_DEV) / sum(case when e.Th_232_D_CONC_BOTTLE_STANDARD_DEV != 0 then 1 else 0 end) end as Th_232_D_CONC_BOTTLE_STANDARD_DEV_Above,case when sum(case when e.Th_234_D_CONC_BOTTLE != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Th_234_D_CONC_BOTTLE) / sum(case when e.Th_234_D_CONC_BOTTLE != 0 then 1 else 0 end) end as Th_234_D_CONC_BOTTLE_Above,case when sum(case when e.Th_234_D_CONC_BOTTLE_STANDARD_DEV != 0 then 1 else 0 end) = 0 then 0.0 else sum(e.Th_234_D_CONC_BOTTLE_STANDARD_DEV) / sum(case when e.Th_234_D_CONC_BOTTLE_STANDARD_DEV != 0 then 1 else 0 end) end as Th_234_D_CONC_BOTTLE_STANDARD_DEV_Above
from Above1 a, E e
where a.Station = e.Station and a.pressureAbove = e.CTDPRS;

-- The next section is for extrapolation onto points
-- with a pressure either above or below but not both
Above2Lib = select Library from Above2;
Below2Lib = select Library from Below2;

AboveNotBelowLib = diff(Above2Lib, Below2Lib);
AboveNotBelow = select a.Library, a.Station, a.pressure,
    a.depthAbove as depth,
    a.tempAbove as temp,
    a.salAbove as sal,
a.OXYGEN_Above as OXYGEN,a.CTDOXY_Above as CTDOXY,a.PHSPHT_Above as PHSPHT,a.SILCAT_Above as SILCAT,a.NITRAT_Above as NITRAT,a.NITRIT_Above as NITRIT,a.TALK_Above as TALK,a.DIC_Above as DIC,a.H2O_2_D_DELTA_BOTTLE_Above as H2O_2_D_DELTA_BOTTLE,a.H2O_18_D_DELTA_BOTTLE_Above as H2O_18_D_DELTA_BOTTLE,a.Al_D_CONC_BOTTLE_Above as Al_D_CONC_BOTTLE,a.STANDARD_DEV_Above as STANDARD_DEV,a.Ba_D_CONC_BOTTLE_Above as Ba_D_CONC_BOTTLE,a.Cd_D_CONC_BOTTLE_Above as Cd_D_CONC_BOTTLE,a.Fe_D_CONC_BOTTLE_Above as Fe_D_CONC_BOTTLE,a.Fe_D_CONC_BOTTLE_FIA_Above as Fe_D_CONC_BOTTLE_FIA,a.Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV_Above as Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV,a.Mn_D_CONC_BOTTLE_Above as Mn_D_CONC_BOTTLE,a.Mn_D_CONC_BOTTLE_STANDARD_DEV_Above as Mn_D_CONC_BOTTLE_STANDARD_DEV,a.Mo_D_CONC_BOTTLE_Above as Mo_D_CONC_BOTTLE,a.Ni_D_CONC_BOTTLE_Above as Ni_D_CONC_BOTTLE,a.Pb_D_CONC_BOTTLE_Above as Pb_D_CONC_BOTTLE,a.U_D_CONC_BOTTLE_Above as U_D_CONC_BOTTLE,a.Zn_D_CONC_BOTTLE_Above as Zn_D_CONC_BOTTLE,a.Y_D_CONC_BOTTLE_Above as Y_D_CONC_BOTTLE,a.La_D_CONC_BOTTLE_Above as La_D_CONC_BOTTLE,a.Pa_231_D_CONC_BOTTLE_Above as Pa_231_D_CONC_BOTTLE,a.Pa_231_D_CONC_BOTTLE_STANDARD_DEV_Above as Pa_231_D_CONC_BOTTLE_STANDARD_DEV,a.Th_230_D_CONC_BOTTLE_Above as Th_230_D_CONC_BOTTLE,a.Th_230_D_CONC_BOTTLE_STANDARD_DEV_Above as Th_230_D_CONC_BOTTLE_STANDARD_DEV,a.Th_232_D_CONC_BOTTLE_Above as Th_232_D_CONC_BOTTLE,a.Th_232_D_CONC_BOTTLE_STANDARD_DEV_Above as Th_232_D_CONC_BOTTLE_STANDARD_DEV,a.Th_234_D_CONC_BOTTLE_Above as Th_234_D_CONC_BOTTLE,a.Th_234_D_CONC_BOTTLE_STANDARD_DEV_Above as Th_234_D_CONC_BOTTLE_STANDARD_DEV
from AboveNotBelowLib d, Above2 a
where d.Library = a.Library;
store(AboveNotBelow, AboveNotBelow);

BelowNotAboveLib = diff(Below2Lib, Above2Lib);
BelowNotAbove = select a.Library, a.Station, a.pressure,
    a.depthBelow as depth,
    a.tempBelow as temp,
    a.salBelow as sal,
a.OXYGEN_Below as OXYGEN,a.CTDOXY_Below as CTDOXY,a.PHSPHT_Below as PHSPHT,a.SILCAT_Below as SILCAT,a.NITRAT_Below as NITRAT,a.NITRIT_Below as NITRIT,a.TALK_Below as TALK,a.DIC_Below as DIC,a.H2O_2_D_DELTA_BOTTLE_Below as H2O_2_D_DELTA_BOTTLE,a.H2O_18_D_DELTA_BOTTLE_Below as H2O_18_D_DELTA_BOTTLE,a.Al_D_CONC_BOTTLE_Below as Al_D_CONC_BOTTLE,a.STANDARD_DEV_Below as STANDARD_DEV,a.Ba_D_CONC_BOTTLE_Below as Ba_D_CONC_BOTTLE,a.Cd_D_CONC_BOTTLE_Below as Cd_D_CONC_BOTTLE,a.Fe_D_CONC_BOTTLE_Below as Fe_D_CONC_BOTTLE,a.Fe_D_CONC_BOTTLE_FIA_Below as Fe_D_CONC_BOTTLE_FIA,a.Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV_Below as Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV,a.Mn_D_CONC_BOTTLE_Below as Mn_D_CONC_BOTTLE,a.Mn_D_CONC_BOTTLE_STANDARD_DEV_Below as Mn_D_CONC_BOTTLE_STANDARD_DEV,a.Mo_D_CONC_BOTTLE_Below as Mo_D_CONC_BOTTLE,a.Ni_D_CONC_BOTTLE_Below as Ni_D_CONC_BOTTLE,a.Pb_D_CONC_BOTTLE_Below as Pb_D_CONC_BOTTLE,a.U_D_CONC_BOTTLE_Below as U_D_CONC_BOTTLE,a.Zn_D_CONC_BOTTLE_Below as Zn_D_CONC_BOTTLE,a.Y_D_CONC_BOTTLE_Below as Y_D_CONC_BOTTLE,a.La_D_CONC_BOTTLE_Below as La_D_CONC_BOTTLE,a.Pa_231_D_CONC_BOTTLE_Below as Pa_231_D_CONC_BOTTLE,a.Pa_231_D_CONC_BOTTLE_STANDARD_DEV_Below as Pa_231_D_CONC_BOTTLE_STANDARD_DEV,a.Th_230_D_CONC_BOTTLE_Below as Th_230_D_CONC_BOTTLE,a.Th_230_D_CONC_BOTTLE_STANDARD_DEV_Below as Th_230_D_CONC_BOTTLE_STANDARD_DEV,a.Th_232_D_CONC_BOTTLE_Below as Th_232_D_CONC_BOTTLE,a.Th_232_D_CONC_BOTTLE_STANDARD_DEV_Below as Th_232_D_CONC_BOTTLE_STANDARD_DEV,a.Th_234_D_CONC_BOTTLE_Below as Th_234_D_CONC_BOTTLE,a.Th_234_D_CONC_BOTTLE_STANDARD_DEV_Below as Th_234_D_CONC_BOTTLE_STANDARD_DEV
from BelowNotAboveLib d, Below2 a
where d.Library = a.Library;
store(BelowNotAbove, BelowNotAbove);

-- Start of interpolation
Both1 = select distinct b.Library, b.Station, b.pressure, a.pressureAbove, b.pressureBelow,
    (a.pressureAbove - a.pressure) / (a.pressureAbove - b.pressureBelow) as mult,
    a.tempAbove, b.tempBelow,
    a.depthAbove, b.depthBelow,
    a.salAbove, b.salBelow,
a.OXYGEN_Above, b.OXYGEN_Below,a.CTDOXY_Above, b.CTDOXY_Below,a.PHSPHT_Above, b.PHSPHT_Below,a.SILCAT_Above, b.SILCAT_Below,a.NITRAT_Above, b.NITRAT_Below,a.NITRIT_Above, b.NITRIT_Below,a.TALK_Above, b.TALK_Below,a.DIC_Above, b.DIC_Below,a.H2O_2_D_DELTA_BOTTLE_Above, b.H2O_2_D_DELTA_BOTTLE_Below,a.H2O_18_D_DELTA_BOTTLE_Above, b.H2O_18_D_DELTA_BOTTLE_Below,a.Al_D_CONC_BOTTLE_Above, b.Al_D_CONC_BOTTLE_Below,a.STANDARD_DEV_Above, b.STANDARD_DEV_Below,a.Ba_D_CONC_BOTTLE_Above, b.Ba_D_CONC_BOTTLE_Below,a.Cd_D_CONC_BOTTLE_Above, b.Cd_D_CONC_BOTTLE_Below,a.Fe_D_CONC_BOTTLE_Above, b.Fe_D_CONC_BOTTLE_Below,a.Fe_D_CONC_BOTTLE_FIA_Above, b.Fe_D_CONC_BOTTLE_FIA_Below,a.Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV_Above, b.Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV_Below,a.Mn_D_CONC_BOTTLE_Above, b.Mn_D_CONC_BOTTLE_Below,a.Mn_D_CONC_BOTTLE_STANDARD_DEV_Above, b.Mn_D_CONC_BOTTLE_STANDARD_DEV_Below,a.Mo_D_CONC_BOTTLE_Above, b.Mo_D_CONC_BOTTLE_Below,a.Ni_D_CONC_BOTTLE_Above, b.Ni_D_CONC_BOTTLE_Below,a.Pb_D_CONC_BOTTLE_Above, b.Pb_D_CONC_BOTTLE_Below,a.U_D_CONC_BOTTLE_Above, b.U_D_CONC_BOTTLE_Below,a.Zn_D_CONC_BOTTLE_Above, b.Zn_D_CONC_BOTTLE_Below,a.Y_D_CONC_BOTTLE_Above, b.Y_D_CONC_BOTTLE_Below,a.La_D_CONC_BOTTLE_Above, b.La_D_CONC_BOTTLE_Below,a.Pa_231_D_CONC_BOTTLE_Above, b.Pa_231_D_CONC_BOTTLE_Below,a.Pa_231_D_CONC_BOTTLE_STANDARD_DEV_Above, b.Pa_231_D_CONC_BOTTLE_STANDARD_DEV_Below,a.Th_230_D_CONC_BOTTLE_Above, b.Th_230_D_CONC_BOTTLE_Below,a.Th_230_D_CONC_BOTTLE_STANDARD_DEV_Above, b.Th_230_D_CONC_BOTTLE_STANDARD_DEV_Below,a.Th_232_D_CONC_BOTTLE_Above, b.Th_232_D_CONC_BOTTLE_Below,a.Th_232_D_CONC_BOTTLE_STANDARD_DEV_Above, b.Th_232_D_CONC_BOTTLE_STANDARD_DEV_Below,a.Th_234_D_CONC_BOTTLE_Above, b.Th_234_D_CONC_BOTTLE_Below,a.Th_234_D_CONC_BOTTLE_STANDARD_DEV_Above, b.Th_234_D_CONC_BOTTLE_STANDARD_DEV_Below
from Below2 b, Above2 a
where b.Station = a.Station and b.pressure = a.pressure;
Store(Both1, SampleToEnvironmental_temporary, [Library]);

,Al_D_CONC_BOTTLE,Ba_D_CONC_BOTTLE,CTDOXY,Cd_D_CONC_BOTTLE,DIC,Fe_D_CONC_BOTTLE,Fe_D_CONC_BOTTLE_FIA,Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV,H2O_18_D_DELTA_BOTTLE,H2O_2_D_DELTA_BOTTLE,La_D_CONC_BOTTLE,Library,Mn_D_CONC_BOTTLE,Mn_D_CONC_BOTTLE_STANDARD_DEV,Mo_D_CONC_BOTTLE,NITRAT,NITRIT,Ni_D_CONC_BOTTLE,OXYGEN,PHSPHT,Pa_231_D_CONC_BOTTLE,Pa_231_D_CONC_BOTTLE_STANDARD_DEV,Pb_D_CONC_BOTTLE,SILCAT,STANDARD_DEV,Station,TALK,Th_230_D_CONC_BOTTLE,Th_230_D_CONC_BOTTLE_STANDARD_DEV,Th_232_D_CONC_BOTTLE,Th_232_D_CONC_BOTTLE_STANDARD_DEV,Th_234_D_CONC_BOTTLE,Th_234_D_CONC_BOTTLE_STANDARD_DEV,U_D_CONC_BOTTLE,Y_D_CONC_BOTTLE,Zn_D_CONC_BOTTLE,depth,pressure,sal,temp
0,42.819500,0.0000,194.8,0.00050,2016.6,1.44,1.532680,0.049760,0.00,0.00,18.250,S0152,3.531710,0.02634,0.000,0.000001,0.015000,2.000,0.0,0.010,0.0,0.0,22.010,1.100000,0.136580,27,2367.2,0.0,0.0,0.0,0.0,0.0000,0.00000,0.0000,138.85001,0.020,12.0,10.0,36.141500,28.409000
1,3.292685,38.1073,255.2,0.01635,2068.5,0.14,0.138050,0.005370,0.18,4.98,20.865,S0243,0.936585,0.00878,144.160,0.200000,0.053333,2.815,210.3,0.090,0.0,0.0,20.170,0.753333,0.043900,10,2310.8,0.0,0.0,0.0,0.0,35.4927,0.80000,12.6078,122.78000,0.165,10.0,2.0,35.038667,15.072333
2,12.614600,42.7746,193.9,0.00130,2078.3,0.10,0.078050,0.000000,0.98,7.41,16.500,S0191,1.658540,0.03707,148.461,0.000001,0.020000,2.470,0.0,0.095,0.0,0.0,16.270,0.445000,0.107320,14,2429.7,0.0,0.0,0.0,0.0,36.6244,0.75122,12.9766,101.04000,0.170,10.0,10.0,37.017001,28.117500
3,1.985365,46.9873,267.7,0.11765,2088.8,0.09,0.080975,0.008295,0.12,0.94,17.265,S0214,0.668295,0.00683,126.356,6.807500,0.100000,3.685,241.9,0.530,0.0,0.0,21.615,3.175000,0.019515,3,2305.2,0.0,0.0,0.0,0.0,21.4829,0.53658,12.4400,119.74000,0.515,10.0,10.0,34.813499,11.465000
4,3.292685,38.1073,255.2,0.01635,2068.5,0.14,0.138050,0.005370,0.18,4.98,20.865,S0001,0.936585,0.00878,144.160,0.200000,0.053333,2.815,210.3,0.090,0.0,0.0,20.170,0.753333,0.043900,10,2310.8,0.0,0.0,0.0,0.0,35.4927,0.80000,12.6078,122.78000,0.165,10.0,10.0,35.038667,15.072333
5,4.419510,48.7639,287.8,0.05280,2073.1,0.52,0.497560,0.019510,-0.26,-1.78,11.050,S0209,0.087800,0.00098,118.365,13.145000,0.165000,4.880,285.6,1.010,0.0,0.0,14.940,0.000001,0.117070,1,2277.8,0.0,0.0,0.0,0.0,25.0829,0.60488,11.6117,114.88000,0.210,10.0,10.0,33.977501,10.093000
6,41.424400,0.0000,203.5,0.00050,0.0,1.77,1.430240,0.020490,0.00,0.00,17.600,S0147,2.760980,0.03805,0.000,0.000001,0.020000,2.010,197.4,0.010,0.0,0.0,23.610,1.000000,0.136580,25,0.0,0.0,0.0,0.0,0.0,0.0000,0.00000,0.0000,138.10001,0.060,10.0,10.0,36.463001,27.220000
7,0.000000,0.0000,228.0,0.00000,2045.8,0.00,0.000000,0.000000,0.00,0.00,0.000,S0237,0.000000,0.00000,0.000,0.000001,0.000001,0.000,230.2,0.070,0.0,0.0,0.000,0.800000,0.000000,7,2334.9,0.0,0.0,0.0,0.0,0.0000,0.00000,0.0000,0.00000,0.000,9.0,2.0,35.369999,20.258000
8,0.000000,0.0000,217.4,0.00000,0.0,0.00,0.000000,0.000000,0.00,0.00,0.000,S0072,0.000000,0.00000,0.000,0.000000,0.000000,0.000,0.0,0.000,0.0,0.0,0.000,0.000000,0.000000,20,0.0,0.0,0.0,0.0,0.0,0.0000,0.00000,0.0000,0.00000,0.000,50.0,40.0,36.667250,19.676625
9,20.400000,41.2976,197.2,0.00170,2002.6,0.34,0.263420,0.019510,0.79,6.76,19.090,S0250,1.629270,0.01854,134.217,0.000001,0.010001,2.140,197.2,0.035,0.0,0.0,20.580,0.365000,0.078050,18,2337.6,0.0,0.0,0.0,0.0,34.7610,0.74146,12.9024,105.47000,3.490,9.0,2.0,35.698501,28.567500


In [98]:
%%query 
Both1 = scan(SampleToEnvironmental_temporary);

Both2 = select Library, Station, pressure,
case when depthBelow = 0.0 then depthAbove when depthAbove = 0.0 then depthBelow else (mult * depthBelow + (1 - mult) * depthAbove) end as depth,
case when tempBelow = 0.0 then tempAbove when tempAbove = 0.0 then tempBelow else 
    (mult * tempBelow + (1 - mult) * tempAbove) end as temp,
case when salBelow = 0.0 then salAbove when salAbove = 0.0 then salBelow else 
    (mult * salBelow + (1 - mult) * salAbove) end as sal,
case when OXYGEN_Below = 0.0 then OXYGEN_Above when OXYGEN_Above = 0.0 then OXYGEN_Below else (mult * OXYGEN_Below + (1 - mult) * OXYGEN_Above) end as OXYGEN,case when CTDOXY_Below = 0.0 then CTDOXY_Above when CTDOXY_Above = 0.0 then CTDOXY_Below else (mult * CTDOXY_Below + (1 - mult) * CTDOXY_Above) end as CTDOXY,case when PHSPHT_Below = 0.0 then PHSPHT_Above when PHSPHT_Above = 0.0 then PHSPHT_Below else (mult * PHSPHT_Below + (1 - mult) * PHSPHT_Above) end as PHSPHT,case when SILCAT_Below = 0.0 then SILCAT_Above when SILCAT_Above = 0.0 then SILCAT_Below else (mult * SILCAT_Below + (1 - mult) * SILCAT_Above) end as SILCAT,case when NITRAT_Below = 0.0 then NITRAT_Above when NITRAT_Above = 0.0 then NITRAT_Below else (mult * NITRAT_Below + (1 - mult) * NITRAT_Above) end as NITRAT,case when NITRIT_Below = 0.0 then NITRIT_Above when NITRIT_Above = 0.0 then NITRIT_Below else (mult * NITRIT_Below + (1 - mult) * NITRIT_Above) end as NITRIT,case when TALK_Below = 0.0 then TALK_Above when TALK_Above = 0.0 then TALK_Below else (mult * TALK_Below + (1 - mult) * TALK_Above) end as TALK,case when DIC_Below = 0.0 then DIC_Above when DIC_Above = 0.0 then DIC_Below else (mult * DIC_Below + (1 - mult) * DIC_Above) end as DIC,case when H2O_2_D_DELTA_BOTTLE_Below = 0.0 then H2O_2_D_DELTA_BOTTLE_Above when H2O_2_D_DELTA_BOTTLE_Above = 0.0 then H2O_2_D_DELTA_BOTTLE_Below else (mult * H2O_2_D_DELTA_BOTTLE_Below + (1 - mult) * H2O_2_D_DELTA_BOTTLE_Above) end as H2O_2_D_DELTA_BOTTLE,case when H2O_18_D_DELTA_BOTTLE_Below = 0.0 then H2O_18_D_DELTA_BOTTLE_Above when H2O_18_D_DELTA_BOTTLE_Above = 0.0 then H2O_18_D_DELTA_BOTTLE_Below else (mult * H2O_18_D_DELTA_BOTTLE_Below + (1 - mult) * H2O_18_D_DELTA_BOTTLE_Above) end as H2O_18_D_DELTA_BOTTLE,case when Al_D_CONC_BOTTLE_Below = 0.0 then Al_D_CONC_BOTTLE_Above when Al_D_CONC_BOTTLE_Above = 0.0 then Al_D_CONC_BOTTLE_Below else (mult * Al_D_CONC_BOTTLE_Below + (1 - mult) * Al_D_CONC_BOTTLE_Above) end as Al_D_CONC_BOTTLE,case when STANDARD_DEV_Below = 0.0 then STANDARD_DEV_Above when STANDARD_DEV_Above = 0.0 then STANDARD_DEV_Below else (mult * STANDARD_DEV_Below + (1 - mult) * STANDARD_DEV_Above) end as STANDARD_DEV,case when Ba_D_CONC_BOTTLE_Below = 0.0 then Ba_D_CONC_BOTTLE_Above when Ba_D_CONC_BOTTLE_Above = 0.0 then Ba_D_CONC_BOTTLE_Below else (mult * Ba_D_CONC_BOTTLE_Below + (1 - mult) * Ba_D_CONC_BOTTLE_Above) end as Ba_D_CONC_BOTTLE,case when Cd_D_CONC_BOTTLE_Below = 0.0 then Cd_D_CONC_BOTTLE_Above when Cd_D_CONC_BOTTLE_Above = 0.0 then Cd_D_CONC_BOTTLE_Below else (mult * Cd_D_CONC_BOTTLE_Below + (1 - mult) * Cd_D_CONC_BOTTLE_Above) end as Cd_D_CONC_BOTTLE,case when Fe_D_CONC_BOTTLE_Below = 0.0 then Fe_D_CONC_BOTTLE_Above when Fe_D_CONC_BOTTLE_Above = 0.0 then Fe_D_CONC_BOTTLE_Below else (mult * Fe_D_CONC_BOTTLE_Below + (1 - mult) * Fe_D_CONC_BOTTLE_Above) end as Fe_D_CONC_BOTTLE,case when Fe_D_CONC_BOTTLE_FIA_Below = 0.0 then Fe_D_CONC_BOTTLE_FIA_Above when Fe_D_CONC_BOTTLE_FIA_Above = 0.0 then Fe_D_CONC_BOTTLE_FIA_Below else (mult * Fe_D_CONC_BOTTLE_FIA_Below + (1 - mult) * Fe_D_CONC_BOTTLE_FIA_Above) end as Fe_D_CONC_BOTTLE_FIA,case when Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV_Below = 0.0 then Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV_Above when Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV_Above = 0.0 then Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV_Below else (mult * Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV_Below + (1 - mult) * Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV_Above) end as Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV,case when Mn_D_CONC_BOTTLE_Below = 0.0 then Mn_D_CONC_BOTTLE_Above when Mn_D_CONC_BOTTLE_Above = 0.0 then Mn_D_CONC_BOTTLE_Below else (mult * Mn_D_CONC_BOTTLE_Below + (1 - mult) * Mn_D_CONC_BOTTLE_Above) end as Mn_D_CONC_BOTTLE,case when Mn_D_CONC_BOTTLE_STANDARD_DEV_Below = 0.0 then Mn_D_CONC_BOTTLE_STANDARD_DEV_Above when Mn_D_CONC_BOTTLE_STANDARD_DEV_Above = 0.0 then Mn_D_CONC_BOTTLE_STANDARD_DEV_Below else (mult * Mn_D_CONC_BOTTLE_STANDARD_DEV_Below + (1 - mult) * Mn_D_CONC_BOTTLE_STANDARD_DEV_Above) end as Mn_D_CONC_BOTTLE_STANDARD_DEV,case when Mo_D_CONC_BOTTLE_Below = 0.0 then Mo_D_CONC_BOTTLE_Above when Mo_D_CONC_BOTTLE_Above = 0.0 then Mo_D_CONC_BOTTLE_Below else (mult * Mo_D_CONC_BOTTLE_Below + (1 - mult) * Mo_D_CONC_BOTTLE_Above) end as Mo_D_CONC_BOTTLE,case when Ni_D_CONC_BOTTLE_Below = 0.0 then Ni_D_CONC_BOTTLE_Above when Ni_D_CONC_BOTTLE_Above = 0.0 then Ni_D_CONC_BOTTLE_Below else (mult * Ni_D_CONC_BOTTLE_Below + (1 - mult) * Ni_D_CONC_BOTTLE_Above) end as Ni_D_CONC_BOTTLE,case when Pb_D_CONC_BOTTLE_Below = 0.0 then Pb_D_CONC_BOTTLE_Above when Pb_D_CONC_BOTTLE_Above = 0.0 then Pb_D_CONC_BOTTLE_Below else (mult * Pb_D_CONC_BOTTLE_Below + (1 - mult) * Pb_D_CONC_BOTTLE_Above) end as Pb_D_CONC_BOTTLE,case when U_D_CONC_BOTTLE_Below = 0.0 then U_D_CONC_BOTTLE_Above when U_D_CONC_BOTTLE_Above = 0.0 then U_D_CONC_BOTTLE_Below else (mult * U_D_CONC_BOTTLE_Below + (1 - mult) * U_D_CONC_BOTTLE_Above) end as U_D_CONC_BOTTLE,case when Zn_D_CONC_BOTTLE_Below = 0.0 then Zn_D_CONC_BOTTLE_Above when Zn_D_CONC_BOTTLE_Above = 0.0 then Zn_D_CONC_BOTTLE_Below else (mult * Zn_D_CONC_BOTTLE_Below + (1 - mult) * Zn_D_CONC_BOTTLE_Above) end as Zn_D_CONC_BOTTLE,case when Y_D_CONC_BOTTLE_Below = 0.0 then Y_D_CONC_BOTTLE_Above when Y_D_CONC_BOTTLE_Above = 0.0 then Y_D_CONC_BOTTLE_Below else (mult * Y_D_CONC_BOTTLE_Below + (1 - mult) * Y_D_CONC_BOTTLE_Above) end as Y_D_CONC_BOTTLE,case when La_D_CONC_BOTTLE_Below = 0.0 then La_D_CONC_BOTTLE_Above when La_D_CONC_BOTTLE_Above = 0.0 then La_D_CONC_BOTTLE_Below else (mult * La_D_CONC_BOTTLE_Below + (1 - mult) * La_D_CONC_BOTTLE_Above) end as La_D_CONC_BOTTLE,case when Pa_231_D_CONC_BOTTLE_Below = 0.0 then Pa_231_D_CONC_BOTTLE_Above when Pa_231_D_CONC_BOTTLE_Above = 0.0 then Pa_231_D_CONC_BOTTLE_Below else (mult * Pa_231_D_CONC_BOTTLE_Below + (1 - mult) * Pa_231_D_CONC_BOTTLE_Above) end as Pa_231_D_CONC_BOTTLE,case when Pa_231_D_CONC_BOTTLE_STANDARD_DEV_Below = 0.0 then Pa_231_D_CONC_BOTTLE_STANDARD_DEV_Above when Pa_231_D_CONC_BOTTLE_STANDARD_DEV_Above = 0.0 then Pa_231_D_CONC_BOTTLE_STANDARD_DEV_Below else (mult * Pa_231_D_CONC_BOTTLE_STANDARD_DEV_Below + (1 - mult) * Pa_231_D_CONC_BOTTLE_STANDARD_DEV_Above) end as Pa_231_D_CONC_BOTTLE_STANDARD_DEV,case when Th_230_D_CONC_BOTTLE_Below = 0.0 then Th_230_D_CONC_BOTTLE_Above when Th_230_D_CONC_BOTTLE_Above = 0.0 then Th_230_D_CONC_BOTTLE_Below else (mult * Th_230_D_CONC_BOTTLE_Below + (1 - mult) * Th_230_D_CONC_BOTTLE_Above) end as Th_230_D_CONC_BOTTLE,case when Th_230_D_CONC_BOTTLE_STANDARD_DEV_Below = 0.0 then Th_230_D_CONC_BOTTLE_STANDARD_DEV_Above when Th_230_D_CONC_BOTTLE_STANDARD_DEV_Above = 0.0 then Th_230_D_CONC_BOTTLE_STANDARD_DEV_Below else (mult * Th_230_D_CONC_BOTTLE_STANDARD_DEV_Below + (1 - mult) * Th_230_D_CONC_BOTTLE_STANDARD_DEV_Above) end as Th_230_D_CONC_BOTTLE_STANDARD_DEV,case when Th_232_D_CONC_BOTTLE_Below = 0.0 then Th_232_D_CONC_BOTTLE_Above when Th_232_D_CONC_BOTTLE_Above = 0.0 then Th_232_D_CONC_BOTTLE_Below else (mult * Th_232_D_CONC_BOTTLE_Below + (1 - mult) * Th_232_D_CONC_BOTTLE_Above) end as Th_232_D_CONC_BOTTLE,case when Th_232_D_CONC_BOTTLE_STANDARD_DEV_Below = 0.0 then Th_232_D_CONC_BOTTLE_STANDARD_DEV_Above when Th_232_D_CONC_BOTTLE_STANDARD_DEV_Above = 0.0 then Th_232_D_CONC_BOTTLE_STANDARD_DEV_Below else (mult * Th_232_D_CONC_BOTTLE_STANDARD_DEV_Below + (1 - mult) * Th_232_D_CONC_BOTTLE_STANDARD_DEV_Above) end as Th_232_D_CONC_BOTTLE_STANDARD_DEV,case when Th_234_D_CONC_BOTTLE_Below = 0.0 then Th_234_D_CONC_BOTTLE_Above when Th_234_D_CONC_BOTTLE_Above = 0.0 then Th_234_D_CONC_BOTTLE_Below else (mult * Th_234_D_CONC_BOTTLE_Below + (1 - mult) * Th_234_D_CONC_BOTTLE_Above) end as Th_234_D_CONC_BOTTLE,case when Th_234_D_CONC_BOTTLE_STANDARD_DEV_Below = 0.0 then Th_234_D_CONC_BOTTLE_STANDARD_DEV_Above when Th_234_D_CONC_BOTTLE_STANDARD_DEV_Above = 0.0 then Th_234_D_CONC_BOTTLE_STANDARD_DEV_Below else (mult * Th_234_D_CONC_BOTTLE_STANDARD_DEV_Below + (1 - mult) * Th_234_D_CONC_BOTTLE_STANDARD_DEV_Above) end as Th_234_D_CONC_BOTTLE_STANDARD_DEV
from Both1; --where Station = 1;

AboveNotBelow = scan(AboveNotBelow);
BelowNotAbove = scan(BelowNotAbove);
Both3 = Both2 + AboveNotBelow + BelowNotAbove;
Store(Both3, SampleToEnvironmental, [Library]);

,Al_D_CONC_BOTTLE,Ba_D_CONC_BOTTLE,CTDOXY,Cd_D_CONC_BOTTLE,DIC,Fe_D_CONC_BOTTLE,Fe_D_CONC_BOTTLE_FIA,Fe_D_CONC_BOTTLE_FIA_STANDARD_DEV,H2O_18_D_DELTA_BOTTLE,H2O_2_D_DELTA_BOTTLE,La_D_CONC_BOTTLE,Library,Mn_D_CONC_BOTTLE,Mn_D_CONC_BOTTLE_STANDARD_DEV,Mo_D_CONC_BOTTLE,NITRAT,NITRIT,Ni_D_CONC_BOTTLE,OXYGEN,PHSPHT,Pa_231_D_CONC_BOTTLE,Pa_231_D_CONC_BOTTLE_STANDARD_DEV,Pb_D_CONC_BOTTLE,SILCAT,STANDARD_DEV,Station,TALK,Th_230_D_CONC_BOTTLE,Th_230_D_CONC_BOTTLE_STANDARD_DEV,Th_232_D_CONC_BOTTLE,Th_232_D_CONC_BOTTLE_STANDARD_DEV,Th_234_D_CONC_BOTTLE,Th_234_D_CONC_BOTTLE_STANDARD_DEV,U_D_CONC_BOTTLE,Y_D_CONC_BOTTLE,Zn_D_CONC_BOTTLE,depth,pressure,sal,temp
0,19.404900,0.000000,216.666667,0.042100,2086.500000,0.090000,0.074150,0.002930,0.000000,0.000000,18.170000,S0159,0.829270,0.004880,0.000000,2.722223,0.225556,2.650000,0.000000,0.187778,0.00000,0.00000,21.610000,1.620000,0.146340,11,2406.300000,0.00000,0.00000,0.00000,0.00000,39.122000,1.951220,0.000000,143.620000,0.050000,74.444444,75.0,36.364111,18.099778
1,6.039020,43.272200,123.785000,0.211600,2176.400000,0.830000,0.741460,0.019510,0.260000,1.610000,17.180000,S0204,0.204880,0.001950,137.491000,20.008000,0.019500,3.580000,0.000000,1.311500,0.00000,0.00000,16.900000,6.889500,0.068290,17,2306.600000,0.00000,0.00000,0.00000,0.00000,42.331700,0.868290,12.915100,103.900000,0.280000,149.100000,150.0,35.333100,13.454450
2,0.000000,0.000000,147.132000,0.000000,2183.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,S0026,0.000000,0.000000,0.000000,20.460000,0.010000,0.000000,0.000000,1.320000,0.00000,0.00000,0.000000,6.140000,0.000000,15,2324.700000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,178.800000,180.0,35.824280,15.119640
3,27.814600,0.000000,194.043802,0.010500,2115.500000,0.300000,0.262440,0.004880,0.000000,0.000000,14.590000,S0104,0.321950,0.009760,0.000000,2.933058,0.020000,2.170000,0.000000,0.113306,0.00000,0.00000,21.330000,1.166529,0.117070,24,2397.600000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,135.850010,0.080000,283.933884,286.0,36.631096,18.848942
4,14.926800,42.213700,195.526667,0.001900,2042.800000,0.110000,0.078050,0.068290,0.890000,7.360000,17.130000,S0200,1.560980,0.029270,136.229000,0.000001,0.023000,2.290000,0.000000,0.087000,0.00000,0.00000,17.570000,0.254000,0.097560,16,2386.800000,0.00000,0.00000,0.00000,0.00000,39.785400,0.839020,12.952200,102.700000,0.070000,49.700000,50.0,36.448166,28.647167
5,27.051762,0.000000,186.789879,0.005394,2111.400000,0.422874,0.365439,0.006906,0.000000,0.000000,14.419028,S0103,0.355246,0.004956,0.000000,2.223482,0.032611,2.153968,0.000000,0.076579,0.00000,0.00000,21.697733,1.009919,0.188527,24,2403.000000,0.00000,0.00000,0.00000,0.00000,50.454068,2.770414,0.000000,132.671377,2.503806,233.344130,235.0,36.706690,19.628664
6,10.680126,41.624400,208.702400,0.002838,2078.240000,0.205920,0.153150,0.002930,0.750000,6.230000,15.630880,S0247,1.499004,0.020162,147.770000,0.014401,0.017400,2.255920,0.000000,0.103440,0.00000,0.00000,14.693120,0.256080,0.076645,15,2419.314400,0.00000,0.00000,0.00000,0.00000,36.368386,0.820605,13.222400,98.007920,0.059040,99.400000,100.0,36.874408,24.253912
7,14.985740,42.213700,203.236364,0.002705,0.000000,0.081818,0.061161,0.041191,0.890000,7.360000,15.040519,S0090,1.219259,0.023376,136.229000,0.040260,0.038052,2.290000,0.000000,0.055844,0.00000,0.00000,18.838182,0.428312,0.105415,16,0.000000,0.00000,0.00000,0.00000,0.00000,35.983310,1.129679,12.952200,116.488961,0.065974,39.740260,40.0,36.490635,24.557935
8,15.219500,0.000000,122.122222,0.123600,2172.529630,0.590000,0.437070,0.006830,0.000000,0.000000,18.040000,S0042,0.331710,0.003900,0.000000,11.655556,0.040000,3.030000,0.000000,0.655926,0.00000,0.00000,18.350000,3.366667,0.224390,35,2385.137037,0.00000,0.00000,0.00000,0.00000,41.463400,1.463410,0.000000,119.310000,0.120000,149.111111,150.0,36.437408,18.943333
9,28.848800,0.000000,204.691369,0.008800,2104.999405,0.230000,0.212680,0.004880,0.000000,0

# Load GC Content conversion table

The following csv file maps 11-mers to the proportion of bases inside it that are 'G' or 'C'.
The file was generated from a standalone Java program, enumerating all 4^11 11-mers.

In [ ]:
%%query
T1 = load("file:///home/gridsan/dhutchison/gits/istc_oceanography/diversity/kmer_gc.csv",
csv(schema(kmer:string,gc_content:float),skip=1,delimiter=","));
store(T1, Kmer11_gc_content);


In [ ]:
%%query
Tgc = scan(Kmer11_gc_content);
Tnorm = scan(kmercounts_combined_scaled);

TsampGC = select n.sampleid, sum(n.norm_cnt * g.gc_content) as gc_content
from Tgc g, Tnorm n
where n.kmer = g.kmer;

store(TsampGC, sample_gc, [sampleid]);

# t-SNE Ordering and Visualization

We used a Matlab script implementing t-SNE to cluster together samples, using the Bray-Curtis distance matrix as input.

By running t-SNE with 1 output dimension and tweaking the perplexity parameter (2.5 is pretty good), we can derive a ordering that places similar samples near each other.

The results of this step are in the following dataset:

In [ ]:
%%query
T1 = load("file:///home/gridsan/dhutchison/gits/istc_oceanography/parse_fastq/tSNE_matlab/t-SNE-BC-1d-labels.csv",
csv(schema(sampleid:string,clusterid:string),skip=0,delimiter=","));
store(T1, t-SNE-BC-1d-labels);

# Join all sample data together

The following queries combine all information for each sample into one relation, for use on the visualization screen.

In [ ]:
%%query
E = scan(SampleToEnvironmental);
S = scan(kmercntnorm_11_forward_stats);
SSUM = scan(kmercnt_11_sum_Psample);
G = scan(sample_gc);
L = scan(t-SNE-BC-1d-labels);
R = select * from E e, S s, G g, L l, SSUM ssum
where e.Library = s.sampleid and e.Library = g.sampleid and e.Library = l.sampleid and s.sampleid = ssum.sampleid;
store(R, SampleToEnvironmental_All);

# Degree Distribution

An easy initial way to characterize a distribution is by its degree distribution.
We plotted the degree distribution of the kmer frequencies of a few samples.
The x-axis is the frequency, and the y-axis is how many kmers occurred with that frequency, normalized by 4^11.

In [ ]:
Tkmer = scan(kmercntnorm_11_forward_Pkmer);

Thist = select sampleid, norm_cnt, count(norm_cnt) as cnt_norm_cnt
from Tkmer
where sampleid = "S0002";

store(Thist, kmer_11_forward_degree_dist);


In [ ]:
#hist = MyriaRelation("kmer_11_forward_degree_dist").to_dataframe()
#hist.plot()

In [ ]:
#TODO: Python to generate these plots below

<img src="kmer-degree-distribution-normalized.png" />

The results surprised us. We did not expect to see such a nice curve. Keep in mind the logarithmic x-axis, which means this is not a normal distribution.

To test whether the degree distribution of the 11-mers was significant or an artifact of a random distribution, we simulated drawing kmers at random in the following figure.  

<img src="kmer-degree-distribution-normalized-withrandom3.png" />

However, we must be careful in how we generate random sequences.
The actual process in which samples are collected is different: we must consider overlapping and duplication.
The same organism can be in the sample more than once (and bacteria can have multiple copies of some of their DNA in plasmids).

We therefore generated a new random sequence by first fixing a long sequence of about 90M base pairs, and sampling from that sequence at random points, restarting at a new random location approcimately every 200 bases (Normal with mean 200, standard deviation 25), until we collect about 900M kmers.

The result is the red triangle distribution below, which matches the empirical distirbutions from real samples more closely.

<img src="kmer-degree-distribution-normalized-withrandom4.png" />

Note that these distributions are very sensitive to k.  We get a very different graph with k=13, particularly because many 13-mers are not present whereas pretty much every 11-mer has a count of at least 1.

# Conclusion

We end with some words on our approach.
Kmers are just one approach to achieve a high goal: identify the species and genome each read came from in a sample.  If we cannot reach genome-level resolution, then at least identify the species or genus.

One method for identifying genomes is sequence alignment a la BLAST and related hidden Markov methods like HMMER.  While BLAST/HMMER has seen many advances---GPU parallelization, breaking the algorithm into stages, etc.---this method does not yet scale to the level of data that we see in our GA02/GA03 datasets or Tara. 

In kmer analysis, we mix up the kmers from all the reads in each sample in order to characterize the sample at a coarser level.  Lower 'k's and GC content give overall characterizations, and higher 'k's may identify particular species, genuses, or genomes.  Someday, we may use a multi-resolution model that bridges between lower 'k's and higher 'k's, or we might start including more alignment information.  To test the viability of any kmer approach, we must ask the following questions:

> What is the strength of a genome's identifying signal in a mixture of kmers, as we add more and more different genomes to the sample?  Surely it is easy to identify a genome from reads of a sample consisting purely of that genome.  What if the sample contains 2 different genomes?  3? 400? 5M? 6T?  Does it depend on how closely related the various genomes are (say, in Hamming/edit distance), or on the abundance of each genome in the ocean sample?

Simulation and signal processing theory can answer these questions, at least with a lower bound.  We ought to know how mixed up a set of samples might be before we cannot distinguish signal from noise.